# Silver Layer: 데이터 정제 및 Risk Score 계산

> **Purpose:** P2T2.bronze의 원시 데이터를 정제, 필터링하여 P2T2.silver에 적재합니다.
> - Null 값 제거 및 이상치 필터링
> - 위험도 스코어(risk_score) 계산 (0.0 ~ 1.0)
> - 중복 레코드 제거
>
> **실제 컬럼명:** `heart_rate`, `systolic_bp`, `diastolic_bp`, `spo2`, `temperature`, `respiratory_rate`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리

In [ ]:

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

## 2. 바이탈 정제 (bronze.vital_signs → silver.cleaned_vital_signs)

In [ ]:

def refine_vital_signs():
    """
    Bronze 바이탈 로그를 정제하여 Silver Layer에 적재합니다.
    
    실제 컬럼: patient_id, timestamp, heart_rate, systolic_bp, 
              diastolic_bp, spo2, temperature, respiratory_rate
    """
    df_bronze = spark.table("P2T2.bronze.vital_signs")
    print(f"Bronze vital_signs: {df_bronze.count():,} rows")
    print(f"Columns: {df_bronze.columns}")
    
    # 1. Null 행 제거
    df_clean = df_bronze.dropna(subset=[
        "heart_rate", "systolic_bp", "diastolic_bp", 
        "spo2", "temperature", "respiratory_rate"
    ])
    
    # 2. 물리적 이상치 필터링
    df_filtered = df_clean.filter(
        (F.col("heart_rate").between(20, 250)) &
        (F.col("systolic_bp").between(50, 300)) &
        (F.col("diastolic_bp").between(20, 200)) &
        (F.col("spo2").between(50, 100)) &
        (F.col("temperature").between(30.0, 45.0)) &
        (F.col("respiratory_rate").between(4, 60))
    )
    
    # 3. 위험도 스코어 계산 (0.0 ~ 1.0)
    df_scored = df_filtered.withColumn(
        "risk_score",
        (
            # 심박수 위험도 (정상 60-100)
            F.when(F.col("heart_rate") > 150, 0.3)
             .when(F.col("heart_rate") > 120, 0.2)
             .when(F.col("heart_rate") > 100, 0.1)
             .when(F.col("heart_rate") < 50, 0.2)
             .otherwise(0.0)
            +
            # 혈압 위험도 (정상 90-140)
            F.when(F.col("systolic_bp") > 180, 0.3)
             .when(F.col("systolic_bp") > 160, 0.2)
             .when(F.col("systolic_bp") > 140, 0.1)
             .when(F.col("systolic_bp") < 80, 0.3)
             .otherwise(0.0)
            +
            # 산소포화도 위험도 (정상 95-100)
            F.when(F.col("spo2") < 85, 0.4)
             .when(F.col("spo2") < 90, 0.3)
             .when(F.col("spo2") < 94, 0.1)
             .otherwise(0.0)
        )
    )
    
    # 4. 중복 제거
    df_dedup = df_scored.dropDuplicates(["timestamp", "heart_rate", "systolic_bp"])
    
    # 5. 처리 시각 추가
    df_final = df_dedup.withColumn("processed_at", F.current_timestamp())
    
    # Silver 저장
    df_final.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("P2T2.silver.cleaned_vital_signs")
    
    silver_cnt = spark.table("P2T2.silver.cleaned_vital_signs").count()
    bronze_cnt = df_bronze.count()
    removed = bronze_cnt - silver_cnt
    print(f"✅ 바이탈 정제 완료")
    print(f"   Bronze: {bronze_cnt:,} → Silver: {silver_cnt:,} ({removed:,} removed)")
    df_final.select("patient_id", "heart_rate", "spo2", "systolic_bp", "risk_score").show(5)

refine_vital_signs()

## 3. 기타 테이블 정제 (DICOM, Emergency, Medical History)

In [ ]:

# DICOM, Emergency, Medical History는 간단 정제 (메타데이터 추가)
for table in ["dicom_metadata", "emergency_data", "medical_history"]:
    df = spark.table(f"P2T2.bronze.{table}")
    df = df.withColumn("processed_at", F.current_timestamp())
    
    target = f"P2T2.silver.cleaned_{table}"
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(target)
    
    cnt = spark.table(target).count()
    print(f"✅ {target}: {cnt:,} rows")

## 4. Silver 품질 리포트

In [ ]:

print("=" * 60)
print("🔍 P2T2.silver 품질 리포트")
print("=" * 60)

# 바이탈 통계
df_vitals = spark.table("P2T2.silver.cleaned_vital_signs")
print(f"\n📊 cleaned_vital_signs: {df_vitals.count():,} rows")

# 위험도 분포
risk_dist = df_vitals.groupBy(
    F.when(F.col("risk_score") >= 0.5, "🔴 Critical")
     .when(F.col("risk_score") >= 0.2, "🟡 Warning")
     .otherwise("🟢 Normal").alias("risk_level")
).count().collect()

for row in risk_dist:
    print(f"   {row['risk_level']}: {row['count']:,}")

# 전체 Silver 테이블
print("\n📋 전체 Silver 테이블:")
for t in spark.sql("SHOW TABLES IN silver").collect():
    name = t['tableName']
    cnt = spark.table(f"silver.{name}").count()
    print(f"   silver.{name}: {cnt:,} rows")

print("=" * 60)